This notebook exists to turn hard to use csvs into simpler ones. It is done with a jupyter notebook because I am lazy

pokemon.csv from https://www.kaggle.com/datasets/rounakbanik/pokemon?resource=download

pokemon with multiple forms that could show up
(list from https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_with_form_differences)
unknown
deoxys
wormadam
rotom**
giratina
shaymin*
arceus*
tornadus
thundurus
landorus
kyurem

In [1]:
# imports:

import numpy as np
import pandas as pd

import string

In [2]:
df = pd.read_csv("pokemon.csv")
df.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
index,,,,,,,,,,,,,,,,,,,,,
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


In [3]:
df.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [4]:
# make new, useful columns
df['sr_gen4'] = .125 * df['against_rock'] # while fairy exists in this gen, no mon changes based on fairy existing.
df['sr_gen5'] = .125 * df['against_rock']
df['dex'] = df['pokedex_number']
df['team_preview_name'] = df['name'].str.lower()
df['battle_name'] = df['name'].str.lower()
useful_df = df[['battle_name', 'sr_gen4', 'sr_gen5', 'team_preview_name', 'dex']].iloc[:649] # up to genesect; end of gen 5 dex
useful_df

,battle_name,sr_gen4,sr_gen5,team_preview_name,dex
index,,,,,
0,bulbasaur,0.1250,0.1250,bulbasaur,1
1,ivysaur,0.1250,0.1250,ivysaur,2
2,venusaur,0.1250,0.1250,venusaur,3
3,charmander,0.2500,0.2500,charmander,4
4,charmeleon,0.2500,0.2500,charmeleon,5
...,...,...,...,...,...
644,landorus,0.1250,0.1250,landorus,645
645,kyurem,0.2500,0.2500,kyurem,646
646,keldeo,0.0625,0.0625,keldeo,647


In [5]:
# add forms
# unknown shows its form on team preview
unown = useful_df.iloc[200]  # pokedex number 201
extra_forms = []
for letter in list(string.ascii_lowercase)[1:] + ["question", "exclamation"]:
    # we can ignore "a" as unknown a is the default form
    form = unown.copy()
    form['battle_name'] = form['battle_name'] + "-" + letter
    form['team_preview_name'] = form['battle_name']
    extra_forms.append(form)

# I assume I can skip castform
# pdon and pyoger don't exist in until ORAS (gen 6). If a gen 6 ubers log ever needs to be done this would be a problem.

# deoxys shows its form on team preview
deoxys = useful_df.iloc[385]  # pokedex number 386
for form_name in ["attack", "defense", "speed"]:  # neutral is base
    form = deoxys.copy()
    form['battle_name'] = form['battle_name'] + "-" + form_name
    form['team_preview_name'] = form['battle_name']
    extra_forms.append(form)

# I assume I can skip burmy
# wormadam has two other forms (sandy and trash). Base form is rock weak, the other forms are neutral
wormadam = useful_df.iloc[412]  # dex number 413
for form_name in ["trash", "sandy"]:  # plant is base
    form = wormadam.copy()
    form['battle_name'] = form['battle_name'] + "-" + form_name
    form['team_preview_name'] = form['battle_name']
    form['sr_gen4'] = form['sr_gen4'] / 2
    form['sr_gen5'] = form['sr_gen5'] / 2
    extra_forms.append(form)
# I assume I can skip cherrim
# I assume I can skip shellos and gastrodon

# Rotom is the most interesting case, as rocks damage changes from gen 4 to gen 5, and per form
rotom = useful_df.iloc[478]  # dex number 479
for form_name, rocks_mul in [("heat", 2), ("wash", 1), ("frost", 2), ("fan", 1), ("mow", 1)]:
    form = rotom.copy()
    form['battle_name'] = form['battle_name'] + "-" + form_name
    form['team_preview_name'] = form['battle_name']
    form['sr_gen5'] = form['sr_gen5'] * rocks_mul
    extra_forms.append(form)

# giritina shows its form on team preview
form = useful_df.iloc[486].copy()  # dex number 487
form['battle_name'] = form['battle_name'] + '-origin'
form['team_preview_name'] = form['battle_name']
extra_forms.append(form)

# shaymin shows its form on team preview. sky form is rock weak
form = useful_df.iloc[491].copy()  # dex number 492
form['battle_name'] = form['battle_name'] + '-sky'
form['team_preview_name'] = form['battle_name']
form['sr_gen4'] = form['sr_gen4'] * 2
form['sr_gen5'] = form['sr_gen5'] * 2
extra_forms.append(form)

# arceus does not show its form on team preview, but does have adjusted rocks damage
useful_df.loc[492, 'team_preview_name'] = useful_df.loc[492, 'team_preview_name'] + "-*"
arceus = useful_df.iloc[492]  # dex number 493
for weak in ["fire", "ice", "flying", "bug"]:
    form = arceus.copy()
    form['battle_name'] = form['battle_name'] + "-" + weak
    form['sr_gen4'] = form['sr_gen4'] * 2
    form['sr_gen5'] = form['sr_gen5'] * 2
    extra_forms.append(form)
for neutral in ["water", "electric", "grass", "poison", "psychic", "rock", "ghost", "dragon", "dark", "fairy"]:
    form = arceus.copy()
    form['battle_name'] = form['battle_name'] + "-" + neutral
    extra_forms.append(form)
for resist in ["fighting", "ground", "steel"]:
    form = arceus.copy()
    form['battle_name'] = form['battle_name'] + "-" + resist
    form['sr_gen4'] = form['sr_gen4'] / 2
    form['sr_gen5'] = form['sr_gen5'] / 2
    extra_forms.append(form)

# I assume I can skip basculin
# I assume I can skip darmanitan
# I assume I can skip deerling and sawsbuck

# genies shows their form on team preview
for ind in [640, 641, 644]:  # dex numbers 641, 642, 645
    form = useful_df.iloc[ind].copy()
    form['battle_name'] = form['battle_name'] + '-therian'  # incarnate are default
    form['team_preview_name'] = form['battle_name']
    extra_forms.append(form)

# kyurem shows its form on team preview
kyurem = useful_df.iloc[645]  # dex number 646
for form_name in ["black", "white"]:
    form = kyurem.copy()
    form['battle_name'] = form['battle_name'] + "-" + form_name  # incarnate are default
    form['team_preview_name'] = form['battle_name']
    extra_forms.append(form)

# I assume I can skip keldeo
# I assume I can skip genesect
# I assume I can skip meloetta

# beyond this is gren and other gen6+ mons, I ignore these

In [9]:
output_df = pd.concat([useful_df, pd.DataFrame(extra_forms)]).sort_values(['dex', 'battle_name']).reset_index(drop=True)
output_df.index.name = "index"
output_df.to_csv(r"../pokemon.csv")
output_df

,battle_name,sr_gen4,sr_gen5,team_preview_name,dex
index,,,,,
0,bulbasaur,0.1250,0.1250,bulbasaur,1
1,ivysaur,0.1250,0.1250,ivysaur,2
2,venusaur,0.1250,0.1250,venusaur,3
3,charmander,0.2500,0.2500,charmander,4
4,charmeleon,0.2500,0.2500,charmeleon,5
...,...,...,...,...,...
705,kyurem-black,0.2500,0.2500,kyurem-black,646
706,kyurem-white,0.2500,0.2500,kyurem-white,646
707,keldeo,0.0625,0.0625,keldeo,647


In [7]:
output_df.iloc[28]

battle_name          nidoran♀
sr_gen4                 0.125
sr_gen5                 0.125
team_preview_name    nidoran♀
dex                        29
Name: 28, dtype: object

,battle_name,sr_gen4,sr_gen5,team_preview_name,dex
index,,,,,
0,bulbasaur,0.1250,0.1250,bulbasaur,1
1,ivysaur,0.1250,0.1250,ivysaur,2
2,venusaur,0.1250,0.1250,venusaur,3
3,charmander,0.2500,0.2500,charmander,4
4,charmeleon,0.2500,0.2500,charmeleon,5
...,...,...,...,...,...
644,landorus,0.1250,0.1250,landorus,645
645,kyurem,0.2500,0.2500,kyurem,646
646,keldeo,0.0625,0.0625,keldeo,647
